In [18]:
import pandas as pd
import numpy as np


In [28]:
pf= pd.read_csv("CUR10MB.csv", low_memory=False)
pf



,identity_line_item_id,identity_time_interval,bill_invoice_id,bill_billing_entity,bill_bill_type,bill_payer_account_id,bill_billing_period_start_date,bill_billing_period_end_date,line_item_usage_account_id,line_item_line_item_type,...,reservation_upfront_value,savings_plan_total_commitment_to_date,savings_plan_savings_plan_a_r_n,savings_plan_savings_plan_rate,savings_plan_used_commitment,savings_plan_savings_plan_effective_cost,savings_plan_amortized_upfront_commitment_for_billing_period,savings_plan_recurring_commitment_for_billing_period,resource_tags_user_environment,resource_tags_user_name
0,nedugc63oxudb6n7rlkjlnxuyzbnxyv6oe7xg37jcpcrzm...,2024-07-01T00:00:00Z/2024-08-01T00:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Tax,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,wjgus2xxzqpt7u3yujueh7dohookrhohoq6pr2pcsszxbu...,2024-07-01T00:00:00Z/2024-08-01T00:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Tax,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,xiyzvwzsxaqzjoo2ubfeom4rewniflq2sxbgwpf3ot6pdg...,2024-07-01T00:00:00Z/2024-08-01T00:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Tax,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,oqt3vope7rnptqhwngvjp7lxx357chjqwnmesunrr5mzer...,2024-07-01T00:00:00Z/2024-08-01T00:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,906829246135,Tax,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,dhql3gqjnkjuek2an2nxnokl76ljxtfy6jjbkspqyw6cfb...,2024-07-01T00:00:00Z/2024-08-01T00:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Tax,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115521,bkhql57jol4np3a2nsfdi2qu5yzmefk3yviciknobywywi...,2024-07-06T11:00:00Z/2024-07-06T12:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Usage,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
115522,mppce22ik2yfxaybkfatml3nlfqm4irk34zlfxw6pjnnot...,2024-07-06T11:00:00Z/2024-07-06T12:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Usage,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
115523,qg4xyz2axlt46zcjym3i4bwwyr6nm33wyex6c3lpbkvbqv...,2024-07-06T11:00:00Z/2024-07-06T12:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Usage,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
115524,hm3eamnhtl5tvkzfqbf4hb4s6fn3sz46skuvaovnlq2l72...,2024-07-06T13:00:00Z/2024-07-06T14:00:00Z,NaN,AWS,Anniversary,713496944077,Mon Jul 01 00:00:00 UTC 2024,Thu Aug 01 00:00:00 UTC 2024,713496944077,Usage,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import psutil
import time
import os

def read_parquet(file_path):
    start_time = time.time()
    process = psutil.Process(os.getpid())

    # Monitor CPU and memory usage
    cpu_percentages = []
    memory_usages = []

    def monitor_resources():
        cpu_percentages.append(process.cpu_percent(interval=0.1))
        memory_usages.append(process.memory_info().rss / (1024 * 1024))  # Convert to MB

    # Read the Parquet file
    monitor_resources()
    df = pd.read_parquet(file_path)
    monitor_resources()

    # Print columns to check their names
    # print("Columns in the Parquet file:", df.columns)

    # Calculate cost details
    try:
        total_cost = df['line_item_blended_cost'].sum()
        tax_cost = df[df['line_item_line_item_description'].str.contains('Tax', case=False, na=False)]['line_item_blended_cost'].sum()
        usage_cost = total_cost - tax_cost
    except KeyError as e:
        print(f"Column not found: {e}")
        return

    cost_details = {
        "cost_details": {
            "total_cost": f"{total_cost:.2f}",
            "tax_cost": f"{tax_cost:.2f}",
            "usage_cost": f"{usage_cost:.2f}"
        }
    }

    end_time = time.time()

    # Calculate average CPU and memory usage
    avg_cpu = sum(cpu_percentages) / len(cpu_percentages)
    avg_memory = sum(memory_usages) / len(memory_usages)

    print("Output:")
    print(cost_details)

    print("\nSystem Configuration and Resource Usage:")
    print(f"Elapsed time: {end_time - start_time:.2f} seconds")
    print(f"Average CPU usage: {avg_cpu:.2f}%")
    print(f"Average Memory usage: {avg_memory:.2f} MB")
    print(f"System configuration: {psutil.virtual_memory().total / (1024 * 1024):.2f} MB RAM")

# Replace with the path to your Parquet file
file_path = "CUR10MB.parquet"

read_parquet(file_path)

Output:
{'cost_details': {'total_cost': '3123.76', 'tax_cost': '476.32', 'usage_cost': '2647.44'}}

System Configuration and Resource Usage:
Elapsed time: 27.14 seconds
Average CPU usage: 0.00%
Average Memory usage: 1193.51 MB
System configuration: 8103.02 MB RAM


In [ ]:
read_parquet(file_path)

Output:
{'cost_details': {'total_cost': '3123.76', 'tax_cost': '476.32', 'usage_cost': '2647.44'}}

System Configuration and Resource Usage:
Elapsed time: 26.56 seconds
Average CPU usage: 0.00%
Average Memory usage: 2130.39 MB
System configuration: 8103.02 MB RAM


In [ ]:
import pandas as pd
import json
import psutil
import time
import os

# Function to extract required data from the parquet file
def extract_aws_cost_usage(parquet_file):
    # Read the necessary columns from the parquet file into a DataFrame
    columns_to_read = [
        'line_item_product_code', 'line_item_unblended_cost', 'line_item_line_item_type',
        'line_item_usage_account_id', 'line_item_usage_type', 'identity_line_item_id',
        'line_item_resource_id'
    ]
    df = pd.read_parquet(parquet_file, columns=columns_to_read, engine='pyarrow', use_threads=True)
    df = df.drop_duplicates()
    print(f"DataFrame shape before filtering: {df.shape}")
    # Filter the data for the required product codes
    product_codes = ['AWSCostExplorer']
    filtered_df = df[df['line_item_product_code'].isin(product_codes)]
     # Print the shape of the DataFrame after filtering
    print(f"Filtered DataFrame shape: {filtered_df.shape}")

    # Print the filtered DataFrame
    if not filtered_df.empty:
        print(filtered_df)
    else:
        print("No data found for the specified product codes.")
    


    parquet_file = 'CUR10MB.parquet'

    extract_aws_cost_usage(parquet_file)

In [22]:
import pandas as pd
import json
import psutil
import time
import os

# Function to extract required data from the parquet file
def extract_aws_cost_usage(parquet_file):
    # Read the necessary columns from the parquet file into a DataFrame
    columns_to_read = [
        'line_item_product_code', 'line_item_unblended_cost', 'line_item_line_item_type',
        'line_item_usage_account_id', 'line_item_usage_type', 'identity_line_item_id',
        'line_item_resource_id'
    ]
    df = pd.read_parquet(parquet_file, columns=columns_to_read, engine='pyarrow', use_threads=True)
    #df = df.drop_duplicates()
    # Filter the data for the required product codes
    product_codes = ['AWSCostExplorer']
    filtered_df = df[df['line_item_product_code'].isin(product_codes)]
    
    # Function to calculate total, usage, and tax costs for a given DataFrame
    def calculate_costs(df):
        total_cost = df['line_item_unblended_cost'].sum()
        usage_cost = df.loc[df['line_item_line_item_type'] != 'Tax', 'line_item_unblended_cost'].sum()
        tax_cost = df.loc[df['line_item_line_item_type'] == 'Tax', 'line_item_unblended_cost'].sum()
        return total_cost, usage_cost, tax_cost

    # Calculate costs for each product
    cost_explorer_costs = calculate_costs(filtered_df[filtered_df['line_item_product_code'] == 'AWSCostExplorer'])
    # data_transfer_costs = calculate_costs(filtered_df[filtered_df['line_item_product_code'] == 'AWSDataTransfer'])
    # elb_costs = calculate_costs(filtered_df[filtered_df['line_item_product_code'] == 'AWSELB'])
    
    # Function to extract region-wise details for a given DataFrame
    def extract_region_wise_details(df):
        region_wise_details = {}
        grouped = df.groupby('line_item_usage_account_id')
        
        for region, group in grouped:
            region_total_cost, region_usage_cost, _ = calculate_costs(group)
            region_details = {}
            for usage_type, usage_group in group.groupby('line_item_usage_type'):
                usage_total_cost = usage_group['line_item_unblended_cost'].sum()
                usage_details = {
                    row['identity_line_item_id']: {
                        'total_cost': row['line_item_unblended_cost'],
                        'identityLineItemId': row['identity_line_item_id'],
                        'line_item_resource_id': row['line_item_resource_id'],
                        'region': row['line_item_usage_account_id'],
                        'line_item_product_code': row['line_item_product_code'],
                        'product_usagetype': row['line_item_usage_type']
                    }
                    for idx, row in usage_group.iterrows()
                }
                region_details[usage_type] = {
                    f"{usage_type}_total_cost": usage_total_cost,
                    f"{usage_type}_details": usage_details
                }
            region_wise_details[region] = [{
                'total_cost': region_total_cost,
                'usage_cost': region_usage_cost,
                'details': region_details
            }]
        
        return region_wise_details

    # Extract region-wise details for each product
    cost_explorer_region_wise_details = extract_region_wise_details(filtered_df[filtered_df['line_item_product_code'] == 'AWSCostExplorer'])
    # data_transfer_region_wise_details = extract_region_wise_details(filtered_df[filtered_df['line_item_product_code'] == 'AWSDataTransfer'])
    # elb_region_wise_details = extract_region_wise_details(filtered_df[filtered_df['line_item_product_code'] == 'AWSELB'])
    
    # Prepare the final output dictionary
    output = {
        "details": {
            "AWSCostExplorer": {
                "total_cost": cost_explorer_costs[0],
                "usage_cost": cost_explorer_costs[1],
                "tax_cost": cost_explorer_costs[2],
                "region_wise_details": cost_explorer_region_wise_details
            }
        }
    }
    
    return output

# Start timer and monitor initial CPU and memory usage
process = psutil.Process(os.getpid())
start_time = time.time()
cpu_percent_start = process.cpu_percent(interval=0.1)
memory_usage_start = process.memory_info().rss / (1024 * 1024)

# Example usage
parquet_file = 'CUR10MB.parquet'
output = extract_aws_cost_usage(parquet_file)

# Convert the output dictionary to a JSON string with double quotes
output_json = json.dumps(output, indent=4)
print(output_json)

# Store the output JSON string to a file
with open('output.json', 'w') as f:
    f.write(output_json)

# Monitor final CPU and memory usage and end timer
cpu_percent_end = process.cpu_percent(interval=0.1)
memory_usage_end = process.memory_info().rss / (1024 * 1024)
end_time = time.time()

# Calculate time taken, average CPU percentage, and average memory usage
time_taken = end_time - start_time
average_cpu_percent = (cpu_percent_start + cpu_percent_end) / 2
average_memory_usage = (memory_usage_start + memory_usage_end) / 2

# Print outputs
print(f"Time taken: {time_taken:.2f} seconds")
print(f"Average CPU percentage: {average_cpu_percent:.2f}%")
print(f"Average memory usage: {average_memory_usage:.2f} MB")
print(f"System configuration: {psutil.virtual_memory().total / (1024 * 1024):.2f} MB RAM")

{
    "details": {
        "AWSCostExplorer": {
            "total_cost": 10.290064516800001,
            "usage_cost": 8.6900645168,
            "tax_cost": 1.6,
            "region_wise_details": {
                "713496944077": [
                    {
                        "total_cost": 10.290064516800001,
                        "usage_cost": 8.6900645168,
                        "details": {
                            "": {
                                "_total_cost": 1.6,
                                "_details": {
                                    "nedugc63oxudb6n7rlkjlnxuyzbnxyv6oe7xg37jcpcrzmdhz55q": {
                                        "total_cost": 0.1,
                                        "identityLineItemId": "nedugc63oxudb6n7rlkjlnxuyzbnxyv6oe7xg37jcpcrzmdhz55q",
                                        "line_item_resource_id": "",
                                        "region": "713496944077",
                                        "line_item_product_

In [25]:
import pandas as pd
import json

def calculate_costs(df):
    total_cost = df['line_item_unblended_cost'].sum()
    usage_cost = df.loc[df['line_item_line_item_type'] != 'Tax', 'line_item_unblended_cost'].sum()
    tax_cost = df.loc[df['line_item_line_item_type'] == 'Tax', 'line_item_unblended_cost'].sum()
    return total_cost, usage_cost, tax_cost

def extract_region_wise_details(df):
    region_wise_details = {}
    grouped = df.groupby('line_item_usage_account_id')

    for region, group in grouped:
        region_total_cost, region_usage_cost, _ = calculate_costs(group)
        region_details = {}

        for usage_type, usage_group in group.groupby('line_item_usage_type'):
            usage_total_cost = usage_group['line_item_unblended_cost'].sum()
            usage_details = {
                row['identity_line_item_id']: {
                    'total_cost': row['line_item_unblended_cost'],
                    'identityLineItemId': row['identity_line_item_id'],
                    'line_item_resource_id': row['line_item_resource_id'],
                    'region': row['line_item_usage_account_id'],
                    'line_item_product_code': row['line_item_product_code'],
                    'product_usagetype': row['line_item_usage_type']
                }
                for idx, row in usage_group.iterrows()
            }
            region_details[usage_type] = {
                f"{usage_type}_total_cost": usage_total_cost,
                f"{usage_type}_details": usage_details
            }
        region_wise_details[region] = [{
            'total_cost': region_total_cost,
            'usage_cost': region_usage_cost,
            'details': region_details
        }]

    return region_wise_details

def main(parquet_file_path, output_file_path):
    # Read the parquet file
    df = pd.read_parquet(parquet_file_path, columns=[
        'line_item_product_code', 'line_item_unblended_cost', 
        'line_item_line_item_type', 'line_item_usage_account_id',
        'line_item_usage_type', 'identity_line_item_id', 
        'line_item_resource_id'
    ])
    
    # Calculate costs for AWSCostExplorer
    filtered_df = df[df['line_item_product_code'] == 'AWSCostExplorer']
    cost_explorer_costs = calculate_costs(filtered_df)
    
    # Extract region-wise details
    cost_explorer_region_wise_details = extract_region_wise_details(filtered_df)
    
    # Prepare output
    output = {
        "details": {
            "AWSCostExplorer": {
                "total_cost": cost_explorer_costs[0],
                "usage_cost": cost_explorer_costs[1],
                "tax_cost": cost_explorer_costs[2],
                "region_wise_details": cost_explorer_region_wise_details
            }
        }
    }
    
    
 
    return output

# Start timer and monitor initial CPU and memory usage
process = psutil.Process(os.getpid())
start_time = time.time()
cpu_percent_start = process.cpu_percent(interval=0.1)
memory_usage_start = process.memory_info().rss / (1024 * 1024)

# Example usage
parquet_file = 'CUR10MB.parquet'
output = extract_aws_cost_usage(parquet_file)

# Convert the output dictionary to a JSON string with double quotes
output_json = json.dumps(output, indent=4)
print(output_json)

# Store the output JSON string to a file
with open('output.json', 'w') as f:
    f.write(output_json)

# Monitor final CPU and memory usage and end timer
cpu_percent_end = process.cpu_percent(interval=0.1)
memory_usage_end = process.memory_info().rss / (1024 * 1024)
end_time = time.time()

# Calculate time taken, average CPU percentage, and average memory usage
time_taken = end_time - start_time
average_cpu_percent = (cpu_percent_start + cpu_percent_end) / 2
average_memory_usage = (memory_usage_start + memory_usage_end) / 2

# Print outputs
print(f"Time taken: {time_taken:.2f} seconds")
print(f"Average CPU percentage: {average_cpu_percent:.2f}%")
print(f"Average memory usage: {average_memory_usage:.2f} MB")
print(f"System configuration: {psutil.virtual_memory().total / (1024 * 1024):.2f} MB RAM")

{
    "details": {
        "AWSCostExplorer": {
            "total_cost": 10.290064516800001,
            "usage_cost": 8.6900645168,
            "tax_cost": 1.6,
            "region_wise_details": {
                "713496944077": [
                    {
                        "total_cost": 10.290064516800001,
                        "usage_cost": 8.6900645168,
                        "details": {
                            "": {
                                "_total_cost": 1.6,
                                "_details": {
                                    "nedugc63oxudb6n7rlkjlnxuyzbnxyv6oe7xg37jcpcrzmdhz55q": {
                                        "total_cost": 0.1,
                                        "identityLineItemId": "nedugc63oxudb6n7rlkjlnxuyzbnxyv6oe7xg37jcpcrzmdhz55q",
                                        "line_item_resource_id": "",
                                        "region": "713496944077",
                                        "line_item_product_

In [32]:
import pandas as pd
import json
import psutil
import time
# This function calculates the output that is required
def calculate_costs(df):
    total_cost = df['line_item_unblended_cost'].sum()
    usage_cost = df[df['line_item_line_item_type'] != 'Tax']['line_item_unblended_cost'].sum()
    tax_cost = df[df['line_item_line_item_type'] == 'Tax']['line_item_unblended_cost'].sum()
    return total_cost, usage_cost, tax_cost


#for extracting region wise detatils of the given product
def extract_region_wise_details(df):
    region_wise_details = {}
    
    grouped_by_region = df.groupby('line_item_usage_account_id')

    for region, region_group in grouped_by_region:
        region_total_cost, region_usage_cost, _ = calculate_costs(region_group)
        region_details = {}

        grouped_by_usage_type = region_group.groupby('line_item_usage_type')

        for usage_type, usage_group in grouped_by_usage_type:
            usage_total_cost = usage_group['line_item_unblended_cost'].sum()
            usage_details = {
                row['identity_line_item_id']: {
                    'total_cost': f"{row['line_item_unblended_cost']:.2f}",
                    'identityLineItemId': row['identity_line_item_id'],
                    'line_item_resource_id': row['line_item_resource_id'],
                    'region': row['line_item_usage_account_id'],
                    'line_item_product_code': row['line_item_product_code'],
                    'product_usagetype': row['line_item_usage_type']
                }
                for _, row in usage_group.iterrows()
            }
            region_details[usage_type] = {
                f"{usage_type}_total_cost": f"{usage_total_cost:.2f}",
                f"{usage_type}_details": usage_details
            }
        
        region_wise_details[region] = {
            'total_cost': f"{region_total_cost:.2f}",
            'usage_cost': f"{region_usage_cost:.2f}",
            'details': region_details
        }

    return region_wise_details

def main(parquet_file_path, output_file_path):
    start_time = time.time()
    process = psutil.Process()
    memory_info_start = process.memory_info()
    cpu_percent_start = process.cpu_percent(interval=None)
    
    #read the parquet file using pyarrow
    df = pd.read_parquet(parquet_file_path, columns=[
        'line_item_product_code', 'line_item_unblended_cost', 
        'line_item_line_item_type', 'line_item_usage_account_id',
        'line_item_usage_type', 'identity_line_item_id', 
        'line_item_resource_id'
    ], engine='pyarrow')
    
    output = {"details": {}}
    
    #filtering the product code column according to the products
    for product_code in df['line_item_product_code'].unique():
        filtered_df = df[df['line_item_product_code'] == product_code]
        
        if not filtered_df.empty:
            product_costs = calculate_costs(filtered_df)
            product_region_wise_details = extract_region_wise_details(filtered_df)
            
            output["details"][product_code] = {
                "total_cost": f"{product_costs[0]:.2f}",
                "usage_cost": f"{product_costs[1]:.2f}",
                "tax_cost": f"{product_costs[2]:.2f}",
                "region_wise_details": product_region_wise_details
            }
    
    with open(output_file_path, 'w') as f:
        json.dump(output, f, indent=4)
    
    memory_info_end = process.memory_info()
    cpu_percent_end = process.cpu_percent(interval=None)
    end_time = time.time()

    time_taken = end_time - start_time
    memory_usage = (memory_info_end.rss - memory_info_start.rss) / (1024 * 1024)  # in MB
    average_cpu_percent = (cpu_percent_start + cpu_percent_end) / 2

    print(f"Time taken: {time_taken:.2f} seconds")
    print(f"Average CPU usage: {average_cpu_percent:.2f}%")
    print(f"Memory usage: {memory_usage:.2f} MB")
    print(f"System memory: {psutil.virtual_memory().total / (1024 * 1024):.2f} MB")

# Usage
parquet_file_path = 'CUR10MB.parquet'  # Replace with your Parquet file path
output_file_path = 'output.json'
main(parquet_file_path, output_file_path)


Time taken: 233.90 seconds
Average CPU usage: 44.45%
Memory usage: 219.90 MB
System memory: 8103.02 MB
